In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

from stimator import read_model
from ipywidgets import interact
%matplotlib inline

styles = ['seaborn-whitegrid', 'seaborn-poster']

# Exemplo: dinâmicas variadas em sinalização celular

## Modelo da resposta do ião cálcio intracelular a estímulos externos 

# (hormonas, fertilização, etc)

### Componentes:

- Recetor de membrana
- IP3 como segundo mensageiro
- Reservatório intracelular de cálcio sensível ao IP3
- Reservatório intracelular de cálcio sensível ao cálcio citosólico
- Transportes ativos, perdas, assimilação de cálcio do exterior, etc

![](images/cicr.png)

![](images/simple_cicr.png)

Neste modelo, o parâmetro $\beta$ representa a intensidade do estímulo, variando no intervalo $[0, 1]$. 

$0$ significa ausência de estímulo  e $1$ significa o estímulo máximo (100%).

&nbsp;

Podemos simular o comportamento deste modelo para diferentes valores de $\beta$.

(Nota: Na declaração do modelo, o parâmetro $\beta$ é representado por `B`.)

In [ ]:
m = read_model("""
title Calcium Spikes
v0         = -> Ca, 1
v1         = -> Ca, k1*B*step(t, 1.0)
k1         = 7.3
B          = 0.1
export     = Ca ->  , 10 ..
leak       = CaComp -> Ca, 1 ..
!! Ca
v2         = Ca -> CaComp, 65 * Ca**2 / (1+Ca**2)
v3         = CaComp -> Ca, 500*CaComp**2/(CaComp**2+4) * Ca**4/(Ca**4 + 0.6561)
init       = state(Ca = 0.1, CaComp = 0.63655)""")

In [ ]:
@interact(B=(0.0, 1.0, 0.01))
def stimulate(B=0.1):
    m.parameters.B = B
    m.solve(tf=8.0, npoints=2000, title='Cytosolic $Ca^{2+}$, $\\beta$ = %g' % B).plot(fig_size = (9,6), yrange=(0,1.5), legend=False, style=styles)

Para os que estão a ler uma versão estática deste notebook, a variação do estímulo ($\beta$) tem o seguinte efeito no comportamento do cálcio citosólico:

In [ ]:
m = read_model("""
title Calcium Spikes
v0         = -> Ca, 1
v1         = -> Ca, k1*B*step(t, 1.0)
k1         = 7.3
B          = 0.1
export     = Ca ->  , 10 ..
leak       = CaComp -> Ca, 1 ..
!! Ca
v2         = Ca -> CaComp, 65 * Ca**2 / (1+Ca**2)
v3         = CaComp -> Ca, 500*CaComp**2/(CaComp**2+4) * Ca**4/(Ca**4 + 0.6561)
init       = state(Ca = 0.1, CaComp = 0.63655)""")

stimulus = (0.0, 0.1, 0.2, 0.25, 0.28, 0.29, 0.3, 0.35, 0.4, 0.45, 0.5, 0.6, 0.75, 0.8, 0.9, 1.0)
titles = ['$\\beta$ = %g' % B for B in stimulus]

s = m.scan({'B':stimulus}, tf=8.0, npoints=1000)

In [ ]:
s.plot(yrange=(0, 1.5), titles=titles,
       legend=False, fig_size=(18.0, 13.0),
       suptitlegend="Dynamics of cytosolic $Ca^{2+}$ as a function of stimulus")